# Assemble a basic vehicle with camera and controller

## Import some libraries

In [3]:
import donkeycar as dk

________             ______                   _________              
___  __ \_______________  /___________  __    __  ____/_____ ________
__  / / /  __ \_  __ \_  //_/  _ \_  / / /    _  /    _  __ `/_  ___/
_  /_/ // /_/ /  / / /  ,<  /  __/  /_/ /     / /___  / /_/ /_  /    
/_____/ \____//_/ /_//_/|_| \___/_\__, /      \____/  \__,_/ /_/     
                                 /____/                              

using donkey v5.0.dev3 ...


## Loading myconfig

In [4]:
cfg = dk.load_config(config_path="/home/pi/mycar/config.py")

INFO:donkeycar.config:loading config file: /home/pi/mycar/config.py
INFO:donkeycar.config:loading personal config over-rides from myconfig.py


In [5]:
V = dk.vehicle.Vehicle()

## Define Camera Helper function

In [1]:
def add_camera(V, cfg):
    inputs = []
    outputs = ['cam/image_array']
    threaded = True
    cam = get_camera(cfg)
    if cam:
        V.add(cam, inputs=inputs, outputs=outputs, threaded=threaded)    

def get_camera(cfg):
    from donkeycar.parts.camera import PiCamera
    cam = PiCamera(image_w=cfg.IMAGE_W, image_h=cfg.IMAGE_H, image_d=cfg.IMAGE_DEPTH,
                   vflip=cfg.CAMERA_VFLIP, hflip=cfg.CAMERA_HFLIP)

    return cam

In [7]:
add_camera(V, cfg, None)

[0:25:13.608686877] [1841]  INFO Camera camera_manager.cpp:299 libcamera v0.0.4+22-923f5d70
INFO:picamera2.picamera2:Initialization successful.
INFO:picamera2.picamera2:Camera now open.
INFO:picamera2.picamera2:Configuration successful!
INFO:picamera2.picamera2:Camera started
INFO:donkeycar.parts.camera:PiCamera opened...
INFO:donkeycar.parts.camera:...warming camera
INFO:donkeycar.parts.camera:PiCamera ready.
INFO:donkeycar.vehicle:Adding part PiCamera.


## Define web controller helper function

In [8]:
from donkeycar.parts.controller import LocalWebController

def add_user_controller(V, cfg, use_joystick, input_image='ui/image_array'):
    """
    Add the web controller and any other
    configured user input controller.
    :param V: the vehicle pipeline.
              On output this will be modified.
    :param cfg: the configuration (from myconfig.py)
    :return: the controller
    """

    #
    # This web controller will create a web server that is capable
    # of managing steering, throttle, and modes, and more.
    #
    ctr = LocalWebController(port=cfg.WEB_CONTROL_PORT, mode=cfg.WEB_INIT_MODE)
    V.add(ctr,
          inputs=[input_image, 'tub/num_records', 'user/mode', 'recording'],
          outputs=['user/steering', 'user/throttle', 'user/mode', 'recording', 'web/buttons'],
          threaded=True)

    return ctr


In [9]:
add_user_controller(V, cfg, None, 'cam/image_array')

INFO:donkeycar.parts.web_controller.web:Starting Donkey Server...
INFO:donkeycar.parts.web_controller.web:You can now go to donkey-52a83a.local:8887 to drive your car.
INFO:donkeycar.vehicle:Adding part LocalWebController.


In [10]:
## Putting them together

V.start(max_loop_count=70)

INFO:donkeycar.vehicle:Starting vehicle at 10 Hz
INFO:tornado.access:304 GET /drive (192.168.3.213) 9.62ms
INFO:tornado.access:101 GET /wsDrive (192.168.3.213) 1.60ms
INFO:donkeycar.parts.web_controller.web:New client connected
INFO:donkeycar.vehicle:Vehicle executed 70 steps in 6.917600631713867 seconds.
INFO:donkeycar.vehicle:Shutting down vehicle and its parts...
INFO:donkeycar.parts.camera:Stopping PiCamera
INFO:picamera2.picamera2:Camera stopped
INFO:picamera2.picamera2:Camera closed successfully.
INFO:donkeycar.vehicle:Part Profile Summary: (times in ms)
INFO:donkeycar.vehicle:
+--------------------+------+------+------+------+------+------+-------+
|        part        | max  | min  | avg  | 50%  | 90%  | 99%  | 99.9% |
+--------------------+------+------+------+------+------+------+-------+
|      PiCamera      | 0.05 | 0.02 | 0.03 | 0.03 | 0.04 | 0.05 |  0.05 |
| LocalWebController | 0.89 | 0.11 | 0.21 | 0.16 | 0.32 | 0.86 |  0.88 |
+--------------------+------+------+------+-

shutting down tornado
shut down tornado completed


(70, 6.917600631713867)

In [33]:
V.stop()

INFO:donkeycar.vehicle:Shutting down vehicle and its parts...
INFO:donkeycar.parts.camera:Stopping PiCamera
INFO:donkeycar.vehicle:Part Profile Summary: (times in ms)
INFO:donkeycar.vehicle:
+--------------------+------+------+------+------+------+------+-------+
|        part        | max  | min  | avg  | 50%  | 90%  | 99%  | 99.9% |
+--------------------+------+------+------+------+------+------+-------+
|      PiCamera      | 0.05 | 0.02 | 0.03 | 0.03 | 0.04 | 0.05 |  0.05 |
| LocalWebController | 0.89 | 0.11 | 0.21 | 0.16 | 0.32 | 0.86 |  0.88 |
+--------------------+------+------+------+------+------+------+-------+


shutting down tornado
shut down tornado completed


In [34]:
!netstat -an -A inet | grep 8887

tcp        2      0 0.0.0.0:8887            0.0.0.0:*               LISTEN     
tcp      942      0 192.168.1.191:8887      192.168.3.213:45878     CLOSE_WAIT 
tcp        0      0 192.168.1.191:8887      192.168.3.213:53022     ESTABLISHED
tcp       58      0 192.168.1.191:8887      192.168.3.213:45896     CLOSE_WAIT 
tcp        1      0 192.168.1.191:8887      192.168.3.213:45882     CLOSE_WAIT 
tcp      857      0 192.168.1.191:8887      192.168.3.213:53016     CLOSE_WAIT 
tcp      883      0 192.168.1.191:8887      192.168.3.213:45894     CLOSE_WAIT 
